In [1]:
import os

from rasterio.plot import reshape_as_image
from rasterio.plot import show
import rasterio.mask
from rasterio.features import rasterize

import pandas as pd
import geopandas as gpd
from shapely.geometry import Polygon
from shapely.ops import cascaded_union

import numpy as np
import cv2

from patchify import patchify
from PIL import Image

import urllib

import xml.etree.ElementTree as ET
import shapely
import geopandas as gpd

from io import BytesIO
from zipfile import ZipFile

from imageio import imread

rs = 42

# Download tile data

In [2]:
url_metadata = "https://www.opengeodata.nrw.de/produkte/geobasis/lusat/dop/dop_jp2_f10/dop_meta.zip"
trgt_filename = 'dop_nw.csv'

response = urllib.request.urlopen(url_metadata)
zipfile = ZipFile(BytesIO(response.read()))

metadata = pd.read_csv(zipfile.open(trgt_filename), 
                       sep=';', 
                       skiprows=5) # skip first 5 rows with irrelevant metadata

metadata.head(10)

,Kachelname,Erfassungsmethode,Aktualitaet,Bildflugnummer,Kamera_Sensor,Bodenpixelgroesse,Spektralkanaele,Koordinatenreferenzsystem_Lage,Koordinatenreferenzsystem_Hoehe,Bezugsflaeche,...,Anzahl_Zeilen,Farbtiefe,Standardabweichung,Dateiformat,Hintergrund,Quelldatenqualitaet,Kompression,Komprimierung,Belaubungszustand,Bemerkungen
0,dop10rgbi_32_375_5666_1_nw_2021,0,2021-06-02,1358/21 Leverkusen Wuppertal,DMCIII-27569_DMCIII,10,RGBI,25832,7837,bDOM,...,10000,8,20,JPEG2000,0,1,1,"GDAL_JP2ECW, 90",3,keine
1,dop10rgbi_32_438_5765_1_nw_2022,0,2022-03-10,1377/22 Greven Ibbenbüren,UCEM3-431S91898X119229-f100_UCE-M3,10,RGBI,25832,7837,bDOM,...,10000,8,20,JPEG2000,0,1,1,"GDAL_JP2ECW, 90",1,keine
2,dop10rgbi_32_366_5723_1_nw_2020,0,2020-03-23,1333/20 Wesel Marl,UCEp-1-31011051_UCEp,10,RGBI,25832,7837,bDOM,...,10000,8,20,JPEG2000,0,1,1,"GDAL_JP2ECW, 90",1,keine
3,dop10rgbi_32_344_5645_1_nw_2021,0,2021-03-02,1355/21 Düsseldorf Kerpen,UCEM3-431S71678X_UCE-M3,10,RGBI,25832,7837,bDOM,...,10000,8,20,JPEG2000,0,1,1,"GDAL_JP2ECW, 90",1,keine
4,dop10rgbi_32_407_5744_1_nw_2022,0,2022-03-03,1379/22 Warendorf,DMCIII-27532_DMCIII,10,RGBI,25832,7837,bDOM,...,10000,8,20,JPEG2000,0,1,1,"GDAL_JP2ECW, 90",1,keine
5,dop10rgbi_32_397_5744_1_nw_2022,0,2022-02-27,1378/22 Bocholt Coesfeld,UCEp-1-31011051-f100_UCEp,10,RGBI,25832,7837,bDOM,...,10000,8,20,JPEG2000,0,1,1,"GDAL_JP2ECW, 90",1,keine
6,dop10rgbi_32_313_5624_1_nw_2021,0,2021-03-07,1356/21 Aachen Kronenburg,UCEM3-1-82416042_UCE-M3,10,RGBI,25832,7837,bDOM,...,10000,8,20,JPEG2000,0,1,1,"GDAL_JP2ECW, 90",1,keine
7,dop10rgbi_32_335_5702_1_nw_2020,0,2020-03-24,1334/20 Duisburg Herne,UCEM3-431S51194X_UCE-M3,10,RGBI,25832,7837,bDOM,...,10000,8,20,JPEG2000,0,1,1,"GDAL_JP2ECW, 90",1,keine
8,dop10rgbi_32_388_5791_1_nw_2022,0,2022-02-23,1376/22 Ahaus Rheine,UCEM3-431S41091X314298-f100_UCE-M3,10,RGBI,25832,7837,bDOM,...,10000,8,20,JPEG2000,0,1,1,"GDAL_JP2ECW, 90",1,keine
9,dop10rgbi_32_304_5671_1_nw_2021,0,2021-02-20,1354/21 Mönchengladbach- Würselen,UCEM3-431S72402X_UCE-M3,10,RGBI,25832,7837,bDOM,...,10000,8,20,JPEG2000,0,1,1,"GDAL_JP2ECW, 90",1,keine


# Retrieving Shapefiles

In [3]:
def get_shapefile(bbox2:tuple, crs='EPSG:25832') -> gpd.GeoDataFrame:
    
    base_url = "https://www.wfs.nrw.de/geobasis/wfs_nw_alkis_vereinfacht?SERVICE=WFS&VERSION=2.0.0&REQUEST=GetFeature&TYPENAMES=ave:GebaeudeBauwerk&BBOX="
    
    x, y = bbox2
    x2 = x + 1000
    y2 = y + 1000
    
    bbox4 = (x, y, x2, y2)
    
    bbox_str = ','.join(list(map(str, bbox4)))
    
    gml_url = ''.join([base_url, bbox_str])
    
    req = urllib.request.Request(gml_url)
    req.get_method = lambda: 'GET'
    response = urllib.request.urlopen(req)
    
    gml_str = response.read()
    
    root = ET.ElementTree(ET.fromstring(gml_str)).getroot()
    
    
    namespace = {'gml': "http://www.opengis.net/gml/3.2",
             'xmlns': "http://repository.gdi-de.org/schemas/adv/produkt/alkis-vereinfacht/2.0",
             'wfs': "http://www.opengis.net/wfs/2.0",
             'xsi': "http://www.w3.org/2001/XMLSchema-instance"
             }
    
    buildings = [i.text for i in root.findall('.//gml:posList', namespace)]
    
    funktions = [i.text for i in root.iter('{http://repository.gdi-de.org/schemas/adv/produkt/alkis-vereinfacht/2.0}funktion')]
    
    ids = [i.items()[0][1] for i in root.findall('.//gml:MultiSurface[@gml:id]', namespace)]
    
    building_shapefiles = []
    
    for id, funktion, build in zip(ids, funktions, buildings):
        coord_iter = iter(build.split(' '))
        coords = list(map(tuple, zip(coord_iter, coord_iter)))
        
        poly = shapely.geometry.Polygon([[float(p[0]), float(p[1])] for p in coords])
        
        building_shapefiles.append({'id': id, 'funktion':funktion, 'geometry': poly})
    
    df = pd.DataFrame.from_records(building_shapefiles)
    gdf = gpd.GeoDataFrame(df, crs=crs)
    
    return gdf

# Combine shapefile to polygon and generate mask

In [4]:
def poly_from_utm(polygon, transform):
    poly_pts = []
    
    poly = shapely.ops.unary_union(polygon)
    for i in np.array(poly.exterior.coords):
        
        # Convert polygons to the image CRS
        poly_pts.append(~transform * tuple(i))
        
    # Generate a polygon object
    new_poly = Polygon(poly_pts)
    return new_poly

def generate_mask(shapefiles, img_url):
    
    with rasterio.open(img_url, "r") as src:
        raster_img = src.read()
        raster_meta = src.meta
    
    # Generate binary mask
    polygons = []
    im_size = (raster_meta["height"], raster_meta["width"])
    for _, row in shapefiles.iterrows():
        if row['geometry'].geom_type == 'Polygon':
            poly = poly_from_utm(row['geometry'], raster_meta["transform"])
            polygons.append(poly)
        else:
            for p in row['geometry']:
                poly = poly_from_utm(p, raster_meta["transform"])
                polygons.append(poly)

    mask = rasterize(shapes=polygons, out_shape=im_size)
    
    return mask, raster_img

# Patchify and save images and masks

In [5]:
def load_and_patchify(img, patch_size, output_path, tile_identifier, num_channels=None):
    
    if num_channels:
        size_x = (img.shape[1]//patch_size) * patch_size
        size_y = (img.shape[2]//patch_size) * patch_size
        
        img = img[:, :size_x, :size_y]
        
        patch_img = patchify(img, (num_channels, patch_size, patch_size), step=patch_size)
        patch_img = np.squeeze(patch_img)
        
    else:
        size_x = (img.shape[0]//patch_size) * patch_size
        size_y = (img.shape[1]//patch_size) * patch_size
    
        img = img[:size_x, :size_y] * 255
        
        patch_img = patchify(img, (patch_size, patch_size), step=patch_size) 
    
    for i in range(patch_img.shape[0]):
        for k in range(patch_img.shape[1]):
            single_patch_img = patch_img[i, k]
            
            path_string = str(tile_identifier) + '_' + str(i) + '_' + str(k) + '.png'
            
            file_path = os.path.join(output_path, path_string)
                
            if num_channels:
                single_patch_img = single_patch_img.swapaxes(0,2)
        
            cv2.imwrite(file_path, single_patch_img)

# Complete pipeline

1. *get_shapefile*:
- **Input**: bounding box values (only north and east, rest is inferred from tile size) as a tuple
- **Output**: geopandas dataframe with polygons of all buildings on the tile

2. *generate_mask*:
- **Input**: geopandas dataframe and tile-image path
- **Output**: mask and image in 1000-1000 pixels

3. *load_and_patchify*:
- **Input**: mask OR image, patch_size (**should correspond to input size for model**), path to output folder (e.g. masks or images), a string identifying each individual 1000-1000 tile (needs to be unique, otherwise output will be overwritten), number of channels (for masks: None, for images: 4)
- **Output**: saves individual images as png files into the specified output folder

# Example

In [6]:
# Download metadata (cell 2)

# index data from metadata dataframe to get coordinates and image link
random_index = np.random.choice(metadata.index.values, 1)

lat = metadata.loc[random_index[0], 'Koordinatenursprung_East']
long = metadata.loc[random_index[0], 'Koordinatenursprung_North']
coords = (lat, long)


base_url = "https://www.opengeodata.nrw.de/produkte/geobasis/lusat/dop/dop_jp2_f10/"
img_path = metadata.loc[random_index[0], 'Kachelname']

# create image url from base url, image url and file extension
img_url = base_url + img_path + '.jp2'

In [12]:
shp_data = get_shapefile(coords)

mask, image = generate_mask(shp_data, img_url)

patch_size = 256

load_and_patchify(image, patch_size, '../output/patchified_imgs/', random_index[0], 4)

load_and_patchify(mask, patch_size, '../output/patchified_masks/', random_index[0])